# Examen de cómputo matricial equipo SVD

**Fecha**

19 de Abril de 2020

**Objetivo**

Programar en el lenguaje R el método de eliminación por bloques para la solución de un sistema de ecuaciones $$Ax=b$$ mediante el método de descomposición en valores singulares (DVS o SVD por siglas en inglés), a través del Algoritmo One-Sided Jacobi.

**Especificaciones de ambiente común de trabajo**

Para trabajar y a efecto de tener un entorno común de trabajo para el desarrollo del proyecto, por favor emplear la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

**Comentarios**

Para mayor información consultar el [Project Board](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/projects/1), la [especificación simplificada del algoritmo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/References/Simplified_SVD_OneSidedJacobi_Algorithm.md) así como las instrucciones de los issues correspondientes


## 1. Funciones auxiliares

Las siguientes funciones serán procedimientos de apoyo en el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 1.1 Generación de índices

In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n (int): número entero postivo 
  #       se refiere al número de columnas de una matriz
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices
}

### 1.2 Verificación de ortogonalidad entre vectores

In [2]:
ortogonal <- function(u,v,TOL=10^-8){
  # Verifica si dos vectores son ortogonales, de acuerdo a cierto nivel de tolerancia, 
  # arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
  #   u (vector): vector de dimension n,
  #   v (vector): vector de dimension n, 
  #   TOL (numeric): real positivo, que sirve como parametro de tolerancia para evaluar ortogonalidad de u y v. 
  #   Notas: 
  #   1) Valor por default TOL es 10^-8
  #   2) Se sugiere una TOL mayor a 10^-32.
  # Returns: 
  #   Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
    if ( norm(u,type ="2") < TOL | norm(v,type ="2") < TOL){ret<-0} 
    else{ 
        if( (u%*%v)/(norm(u,type ="2")*norm(v,type ="2")) < TOL){ret<-1}
        else{ret<-0}  
    }
  ret
}

### 1.3 Función signo

In [3]:
signo<-function(x) {
  # Indica el signo de un número x
  # Args: 
  #    x (numeric): número a revisar
  # Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

### 1.4 Solver dada descomposición SVD

In [4]:
solver <- function(U,S,V,b){
    # Construye la solución de un sistema de ecuaciones a partir de matrices 
    # U, S, V, y vector b. Se asume que S es diagonal. 
    # Para ello resuelve S d = U^Tb, y construye x=Vd.
    # Notas:
    # 1) Se utilizó la función backsolve para resolver el sistema triangular.
    # 2) Al ser S diagonal, es indistinto si usar un solver para matrices traingulares inferiores o superiores.
    # Args: 
    #     U (matriz): matriz para lado derecho de sistema S d = U^Tb, con entrada reales y dimension m x n,
    #     S (matriz): matriz diagonal, que define sistema sistema S d = U^Tb, con entrada reales y dimension n x n,
    #     V (matriz): para construir x, con entrada reales y dimension n x n, 
    #     b (vector): vector con el que se forma lado derecho de primer sistema, de dimension m.
    # Returns: 
    #     x (vector): vector formado por la solucion de S d = U^Tb, multiplicado por V, con dimension n
  d = backsolve(S, t(U)%*%b)
  x = V%*%d
  return(x)
}

## 2. Algoritmo SVD y solución de sistema lineal

Esta sección aborda el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 2.1 One-sided Jacobi numerical aproximación

In [11]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    # Calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representa a la matriz S(mxm) matriz diagonal,el segundo a la matriz U(nxm)
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    # Nota: Esta función estima la SVD thin,la cual calcula unicamente las m columnas de U correspondientes a los m renglones de V. De esta
    # manera las columnas restantes de U no son calculadas, provocando una mejora significativa en velocidad de ejecución comparada con la 
    # la Full SVD. Referencia: https://en.wikipedia.org/wiki/Singular_value_decomposition#Thin_SVD.
    
    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicializa valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0
    stop<-FALSE
    
    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
    for(i in 1:(n-1)){
    for(j in (i+1):n){
      col_j<-ak[,j]
      col_i<-ak[,i]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL)==1){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-col_i%*%col_i
        b<-col_j%*%col_j
        c<-col_i%*%col_j
        
        #si c es cercano a cero no actualiza
        if(c<TOL){
            stop<-TRUE
            break}
          
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        temp<-ak[,i] 
        ak[,i]<-c(cs)*temp-c(sn)*ak[,j]
        ak[,j]<-c(sn)*temp+c(cs)*ak[,j]
        
        
        #actualiza las columnas de la matriz vk
        temp<-vk[,i] #cambio
        vk[,i]<-c(cs)*temp-c(sn)*vk[,j]
        vk[,j]<-c(sn)*temp+c(cs)*vk[,j]             
       }#cierra else
        }#cierra for j
      if(stop==TRUE){
        stop<-FALSE
        break
         }
        }#cierra for i
  k<-k+1
 }#cierra while
    
    #Obtener sigma
    sig<-apply(ak, 2, function(x){norm(x,"2")})

    #Obtener U
    for(i in 1:n){
        if (sig[i]<TOL){
            uk[,i]<-0  
        } else{
        uk[,i] <- ak[,i]/sig[i]
        }
    }

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    index <- order(sig,decreasing = TRUE)
    vk <- vk[,index]
    S <- diag(sig[index])
    uk <- uk[,index]

    list(S = S, U = uk, V= vk)
 }   

In [16]:
#Ejemplo
#parametros de entrada
A<-matrix(c(7,0,1,-5.1,0.4,1,3,5,8,-1,6,-1,7,0,1,-5.1),nrow=4)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [18]:
svd$U %*% svd$S %*% t(svd$V)

7.000000e+00,0.4,8,7.000000e+00
-1.150263e-11,1.0,-1,1.150241e-11
1.000000e+00,3.0,6,1.000000e+00
-5.100000e+00,5.0,-1,-5.100000e+00


### 2.2 Linear solver aproximating SVD decomposition using One-sided Jacobi algorithm

In [8]:
sel_solver<-function(A,b,TOL=10**-8,maxsweep=20){
    # Aproxima la solución de un sistema de ecuaciones lineales (SEL) de la forma Ax=b 
    # usando la descomposición SVD de A obtenido por medio del método de One-sided Jacobi.
    # Args: 
    #    A (matriz): matriz de coeficientes del SEL, de números reales y de dimension m x n.
    #    b (vector): vector de lado derecho del sistema, de números reales y de dimension m.
    #    TOL (numeric): real positivo, que sirve para controlar la convergencia del metodo iterativo. 
    #             Nota: se usa un valor default de 10^-8 como tolerancia
    #    maxsweep (int): numero entero posiitivo, que se usar para controlar el número máximo de sweeps
    #                    que se ejecutan como parte del metodo iterativo.
    # Returns: 
    #    x (vector): vector solución del SEL, con dimension n.

    svd<-svd_jacobi_aprox(A,TOL,maxsweep)
    x<-solver(svd$U,svd$S,svd$V,b)
    x
}

In [9]:
A<-matrix(c(-1,3,2,2,1,3),nrow=3)
b<-c(5,7,12)
x<-sel_solver(A,b,maxsweep=40)

In [10]:
A%*%x

5
7
12


### 2.3 Eliminación por bloques basada en solver linear que usa SVD

In [11]:
bloques<-function(A,b,corte) {
  # Dadas matriz A de n x ny un vector b de dimension n, así como un parametro corte (de dimension), 
  # genera una descomposicion en 4 y 2 bloques de estos.
  # En concreto, el parametro corte sirve para dividir A en los bloques de las siguientes dimensiones
  #    A11: bloque dado por A[1:corte,1:corte] (cuadrante superior izquierdo de A)
  #    A12: bloque dado por A[1:corte, corte+1:n] (cuadrante superior derecho de A)
  #    A21: bloque dado por A[corte+1:n,1:corte] (cuadrante inferior izquierdo de A)
  #    A22: bloque dado por A[corte+1:n,corte+1:n] (cuadrante inferior derecho de A)
  # Asimismo, b se divide como:
  #    b1: primeras corte-entradas de b
  #    b2: entradas restantes de b
    
  # Args: 
  #    A (matriz): matriz de dimensiones n x n sobre la que se hara la division en bloques, 
  #    b (vector): vector de dimensiones n sobre el cual que se hara la division en bloques,
  #    corte (int): entero positivo, que se usa como parametro de dimension con respecto al cual se hace la division de A (valor entre 1 y n)
  #Returns: 
  #     lista con la matriz A dividida en 4 bloques (lista): A11,A12, A21, A22 y el vector b dividido en 2 bloques b1, b2
  #     Notas: dimensiones de los bloques 
  #       A11 (corte x corte), A12 (corte x n-corte), A11 (n-corte x corte), A22 (n-corte x n-corte)
  #       b1 (corte) y b2 (n-corte)
    
    
  # Obtiene la descomposicion en bloques de A, segun parametro corte
  a11 <- A[c(1:corte),c(1:corte)]
  a12 <- A[c(1:corte),c((corte+1):dim(A)[2])]
  a21 <- A[c((corte+1):dim(A)[1]),c(1:corte)]
  a22 <- A[c((corte+1):dim(A)[1]),c((corte+1):dim(A)[2])]
    
  # Obtiene la descomposicion en bloques de b, segun parametro corte
  b1 <- b[1:(corte)]
  b2 <- b[((corte)+1):length(b)]

  list(A11 = a11,A12 =a12, A21=a21, A22=a22, b1 = b1, b2 = b2)
} 

In [12]:
#Prueba
A = matrix(sample(0:50,7*7,replace=TRUE), c(7,7)) 
b = c(1:dim(A)[1])

A
b
bloques(A,b,4)


46,48,34,19,42,50,39
33,26,1,42,50,12,22
38,50,7,45,50,15,2
47,50,41,9,1,26,46
8,0,5,22,30,0,29
42,32,7,16,14,35,1
8,11,6,32,29,19,43


[1] 1 2 3 4 5 6 7

$A11
     [,1] [,2] [,3] [,4]
[1,]   46   48   34   19
[2,]   33   26    1   42
[3,]   38   50    7   45
[4,]   47   50   41    9

$A12
     [,1] [,2] [,3]
[1,]   42   50   39
[2,]   50   12   22
[3,]   50   15    2
[4,]    1   26   46

$A21
     [,1] [,2] [,3] [,4]
[1,]    8    0    5   22
[2,]   42   32    7   16
[3,]    8   11    6   32

$A22
     [,1] [,2] [,3]
[1,]   30    0   29
[2,]   14   35    1
[3,]   29   19   43

$b1
[1] 1 2 3 4

$b2
[1] 5 6 7

Algoritmo

Sean $A$ y $A_{11}$ no singulares.

1) Calcular $A_{11}^{-1}A_{12}$ y $A_{11}^{-1}b_1$ teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales:

Para realizar la multiplicación $A_{11}^{-1}b_1$ definimos $y=A_{11}^{-1}b_1$ y por tanto $A_{11}y = b_1$ ($A_{11}$ es no singular). Así, resolvemos para $y$ el sistema anterior y habremos calculado $A_{11}^{-1}b_1$. Similarmente definimos $Y=A_{11}^{-1}A_{12}$ con lo que se tiene $A_{11}Y=A_{12}$. Resolvemos para $Y \in \mathbb{R} ^{n_1 \times n_1}$ y habremos calculado $A_{11}^{-1}A_{12}$.
2) Calcular el complemento de Schur del bloque $A_{11}$ en $A$: $S = A_{22}-A_{21}A_{11}^{-1}A_{12}$. Calcular $ \hat{b} = b_2-A_{21}A_{11}^{-1}b_1$.

3) Resolver $Sx_2 = \hat{b}$.

4) Resolver $A_{11}x_1 = b_1-A_{12}x_2$.

In [13]:
install.packages("matrixcalc")
library("matrixcalc")


The downloaded binary packages are in
	/var/folders/4h/pz7sf1h93mn0vsyplpp3jxx40000gn/T//RtmpNuUwE1/downloaded_packages


In [14]:
eliminacion_bloques <- function(A,b,corte, TOL, maxsweep){
  # Aproxima la solución de un sistema de ecuaciones lineales (SEL) de la forma Ax=b, usando el método
  # de eliminación por bloques. La solución de los sistemas lineales inducidos por el complemento de Schur
  # se aproxima usando la descomposicion SVD aproximada con el algoritmo One-Sided Jacobi
  #Args: 
  #    A (matriz):  matriz de dimensiones n x n, asociada al sistema del sistema Ax=b que se dividira en bloques para su solucion
  #          Nota: para que exista solucion A y el el bloque A11 deben ser no singulares
  #    b (vector): vector de dimension n, asociado al sistema del sistema Ax=b que se dividira en bloques para su solucion
  #    corte (int): entero positivo, que se usara como parametro para divir en bloques a A y B, (valor entre 1 y n)
  #    TOL (numeric): real positivo, que sirve para controlar la convergencia del algoritmo One-Sided Jacobi para estimar SVD
  #    maxsweep (int): numero entero positivo, que sirve como parametro del máximo de sweeps del algoritmo One-Sided Jacobi
  #                    para estimar SVD (valor entre 1 y (n-1)*n/2)
  #Returns: 
  #    x: vector de dimensiones n x n

  # Descomposicion en bloques A11, A12, A21, A22, b1 y b2, segun parametro de dimension
  bloq = bloques(A,b,corte)

  # Evalua la singularidad de A y el bloque A11, en caso de que una de tales sea singular
  # arroja mensaje al usuario, en caso contrario intenta aproximar sistema inducidos por 
  # complemento de Schur
  if(is.singular.matrix(A,TOL) ==FALSE & is.singular.matrix(bloq$A11,TOL) == FALSE){

  # Solucion del sistema A11 y = b1, usando algoritmo One-Sided Jacobi
  y = sel_solver(bloq$A11,bloq$b1,TOL, maxsweep)
      
  # Solucion del sistema A11 Y = A12 
  Y= bloq$A12
  for(i in 1:dim(bloq$A12)[2]){
    Y[,i] = sel_solver(bloq$A11,bloq$A12[,i],TOL, maxsweep)
  }
      
  # Construye matrix S = A22 A11^-1 A12
  S = bloq$A22 - bloq$A21%*%Y

 # Construye b_hat = b2 - A21 A11^-1 b1
  b_hat = bloq$b2-bloq$A21%*%y
      
  # Solucion del sistema S x2 = b_hat algoritmo One-Sided Jacobi
  x_2 = sel_solver(S,b_hat,TOL, maxsweep)

  # Construye b_hat2 S x2 = b1 - A12 b_hat
  b_hat2 = bloq$b1 -bloq$A12%*%x_2

  # Solucion del sistema A11 x2 = b_hat algoritmo One-Sided Jacobi
  x_1 = sel_solver(bloq$A11,b_hat2,TOL, maxsweep)
      
  # Solucion del sistema original
  x <- c(x_1,x_2)
  x
  }else{
    print("Singularidad de las matrices involucradas; no hay solucion")
  }
  
}

In [15]:
A = matrix(sample(0:50,7*7,replace=TRUE), c(7,7)) 
b = c(1:dim(A)[1])
TOL = 10**-8
maxsweep=40
corte = 4
eliminacion_bloques(A,b,corte, TOL, maxsweep)

[1]  0.06447515 -7.09675733  3.62280314  1.06816512  2.65850158 -1.20113910
[7] -1.28871239